In [1]:
import numpy as np
import pandas as pd
import csv
import re
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, _tree
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
!pip install pyttsx3

In [3]:
!sudo apt-get install espeak

'sudo' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
import pyttsx3

In [5]:
engine = pyttsx3.init()

In [6]:
def text_to_speech(text):
    engine.setProperty('rate', 150)    # Speed percent (can go over 100)
    engine.setProperty('volume', 0.9)  # Volume 0-1
    engine.say(text)
    engine.runAndWait()

In [7]:
training = pd.read_csv('Training.csv')
testing= pd.read_csv('Testing.csv')

In [8]:
shape = training.shape
print("Shape of Training dataset: ", shape)

Shape of Training dataset:  (4920, 133)


In [9]:
training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4920 entries, 0 to 4919
Columns: 133 entries, itching to prognosis
dtypes: int64(132), object(1)
memory usage: 5.0+ MB


In [10]:
null_values_count = training.isnull().sum()
null_values_count

itching                 0
skin_rash               0
nodal_skin_eruptions    0
continuous_sneezing     0
shivering               0
                       ..
inflammatory_nails      0
blister                 0
red_sore_around_nose    0
yellow_crust_ooze       0
prognosis               0
Length: 133, dtype: int64

In [11]:
training.head(500)

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Gastroenteritis
496,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Gastroenteritis
497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Gastroenteritis
498,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Gastroenteritis


In [12]:
cols= training.columns
cols= cols[:-1]
x = training[cols]
y = training['prognosis']

In [13]:
reduced_data = training.groupby(training['prognosis']).max()
reduced_data.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,pus_filled_pimples,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze
prognosis,,,,,,,,,,,,,,,,,,,,,
(vertigo) Paroymsal Positional Vertigo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AIDS,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Acne,0,1,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
Alcoholic hepatitis,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Allergy,0,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

In [15]:
y

array([15, 15, 15, ..., 38, 35, 27])

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
testx    = testing[cols]
testy    = testing['prognosis']
testy    = le.transform(testy)

In [17]:
clf1  = DecisionTreeClassifier()
clf = clf1.fit(x_train,y_train)
scores = cross_val_score(clf, x_test, y_test, cv=3)
print("Mean Score: ",scores.mean())

Mean Score:  0.9698271844086278


In [18]:
model=SVC()
model.fit(x_train,y_train)
print("Accuracy score for svm: ", model.score(x_test,y_test))
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
features = cols

Accuracy score for svm:  1.0


In [19]:
severityDictionary=dict()
description_list = dict()
precautionDictionary=dict()
symptoms_dict = {}
for index, symptom in enumerate(x):
       symptoms_dict[symptom] = index

def calc_condition(exp,days):
    sum=0
    for item in exp:
         sum=sum+severityDictionary[item]
    if((sum*days)/(len(exp)+1)>13):
        print("You should take the consultation from doctor. ")
    else:
        print("It might not be that bad but you should take precautions.")


def getDescription():
    global description_list
    with open('symptom_Description.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            _description={row[0]:row[1]}
            description_list.update(_description)

            
def getSeverityDict():
    global severityDictionary
    with open('Symptom_severity.csv') as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        try:
            for row in csv_reader:
                _diction={row[0]:int(row[1])}
                severityDictionary.update(_diction)
        except:
            pass

def getprecautionDict():
    global precautionDictionary
    with open('symptom_precaution.csv') as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            _prec={row[0]:[row[1],row[2],row[3],row[4]]}
            precautionDictionary.update(_prec)

In [20]:
def getInfo():
    print("-----------------------------------HealthCare ChatBot-----------------------------------")
    print("\nYour Name? \t\t\t\t",end="->")
    name=input("")
    print("Hello", name)

In [21]:
def check_pattern(dis_list,inp):
    pred_list=[]
    inp=inp.replace(' ','_')
    patt = f"{inp}"
    regexp = re.compile(patt)
    pred_list=[item for item in dis_list if regexp.search(item)]
    if(len(pred_list)>0):
        return 1,pred_list
    else:
        return 0,[]

In [22]:
def sec_predict(symptoms_exp):
    df = pd.read_csv('Training.csv')
    X = df.iloc[:, :-1]
    y = df['prognosis']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)
    rf_clf = DecisionTreeClassifier()
    rf_clf.fit(X_train, y_train)

    symptoms_dict = {symptom: index for index, symptom in enumerate(X)}
    input_vector = np.zeros(len(symptoms_dict))
    for item in symptoms_exp:
      input_vector[[symptoms_dict[item]]] = 1

    return rf_clf.predict([input_vector])

In [23]:
def print_disease(node):
    node = node[0]
    val  = node.nonzero()
    disease = le.inverse_transform(val[0])
    return list(map(lambda x:x.strip(),list(disease)))

In [24]:
import pyttsx3
from sklearn.tree import _tree

# Initialize the text-to-speech engine
engine = pyttsx3.init()

# Function to traverse the decision tree and make predictions
def tree_to_code(tree, feature_names):
    # Get the underlying tree structure from the classifier
    tree_ = tree.tree_

    # Map feature indices to feature names for easy reference
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    # Prepare a list of symptom names for validation
    chk_dis = ",".join(feature_names).split(",")
    symptoms_present = []  # List to hold symptoms present during the recursion

    while True:
        # Use text-to-speech to prompt the user for input
        engine.say("Enter the symptom you are experiencing.")
        engine.runAndWait()

        # Get the user input for symptoms
        print("Enter the symptom you are experiencing: ", end="-> ")
        disease_input = input("")  # Read user input

        # Check if the input matches any known symptoms
        conf, cnf_dis = check_pattern(chk_dis, disease_input)
        if conf == 1:  # If a match is found
            print("Searches related to input:")
            for num, it in enumerate(cnf_dis):
                print(num, ")", it)  # Display matching symptoms

            # If multiple symptoms are found, ask the user to select one
            if num != 0:
                print(f"Select the one you meant (0 - {num}): ", end="")
                conf_inp = int(input(""))  # Get the user's selection
            else:
                conf_inp = 0  # Default selection if only one match is found

            # Update disease_input to the confirmed symptom
            disease_input = cnf_dis[conf_inp]
            break  # Exit the loop after valid input
        else:
            print("Enter a valid symptom.")  # Prompt for valid input

    while True:
        try:
            # Ask the user how many days they've been experiencing the symptoms
            num_days = int(input("Okay. From how many days? : "))  # Read the number of days
            break  # Exit loop on valid input
        except ValueError:
            print("Enter a valid input.")  # Prompt for valid input if there's an error

    # Recursive function to traverse the decision tree
    def recurse(node, depth):
        indent = "  " * depth  # Create indentation based on depth for clarity
        if tree_.feature[node] != _tree.TREE_UNDEFINED:  # Check if node is a decision node
            name = feature_name[node]  # Get the feature name
            threshold = tree_.threshold[node]  # Get the threshold for the decision

            # Determine if the input symptom matches the feature
            val = 1 if name == disease_input else 0

            # Recursively traverse left or right based on the threshold
            if val <= threshold:
                recurse(tree_.children_left[node], depth + 1)  # Traverse left subtree
            else:
                symptoms_present.append(name)  # Record the symptom
                recurse(tree_.children_right[node], depth + 1)  # Traverse right subtree
        else:
            # If a leaf node is reached, get the predicted disease
            present_disease = print_disease(tree_.value[node])

            # Print the predicted disease at this step
            print(f"{indent}Predicted disease at this step: {present_disease[0]}")

            # Get relevant symptoms for the predicted disease
            red_cols = reduced_data.columns
            symptoms_given = red_cols[reduced_data.loc[present_disease].values[0].nonzero()]

            # Use text-to-speech to ask the user about additional symptoms
            engine.say("Are you experiencing any symptoms?")
            engine.runAndWait()
            print("Are you experiencing any:")
            symptoms_exp = []  # List to hold additional symptoms

            # Ask the user about each of the relevant symptoms
            for syms in list(symptoms_given):
                inp = ""
                engine.say(f"{syms}, are you experiencing it?")
                engine.runAndWait()
                print(f"{syms}? : ", end='')
                while True:
                    inp = input("")  # Get user input
                    if inp in ["yes", "no"]:  # Validate input
                        break
                    else:
                        print("Provide proper answers (yes/no): ", end="")
                if inp == "yes":
                    symptoms_exp.append(syms)  # Add to symptoms list if the answer is yes

                # After each symptom inquiry, perform prediction
                second_prediction = sec_predict(symptoms_exp)  # Perform a second prediction
                print(f"Current prediction based on your symptoms: {present_disease[0]} or {second_prediction[0]}")

            # Evaluate the condition based on symptoms and number of days
            calc_condition(symptoms_exp, num_days)
            if present_disease[0] == second_prediction[0]:  # Compare predictions
                # If both predictions match
                engine.say(f"You may have {present_disease[0]}.")
                engine.runAndWait()
                print(f"You may have {present_disease[0]}.")
                print(description_list[present_disease[0]])  # Show disease description

            else:
                # If predictions differ, inform the user
                engine.say(f"You may have {present_disease[0]} or {second_prediction[0]}.")
                engine.runAndWait()
                print(f"You may have {present_disease[0]} or {second_prediction[0]}.")
                print(description_list[present_disease[0]])  # Show description of first disease
                print(description_list[second_prediction[0]])  # Show description of second disease

            # Display precautions for the predicted disease
            precaution_list = precautionDictionary[present_disease[0]]
            print("Take the following measures:")
            for i, j in enumerate(precaution_list):  # List precautions
                print(i + 1, ")", j)

    # Start the recursion from the root of the tree
    recurse(0, 1)

# Functions to load severity, description, precautions, and user information
getSeverityDict()  # Load severity data
getDescription()   # Load symptom descriptions
getprecautionDict()  # Load precaution measures
getInfo()  # Gather user's name and greeting
tree_to_code(clf, cols)  # Call the tree_to_code function with the classifier and features

print("----------------------------------------------------------------------------------------------------------------------------------")


-----------------------------------HealthCare ChatBot-----------------------------------

Your Name? 				->

Hello 
Enter the symptom you are experiencing: -> Searches related to input:
0 ) joint_pain
1 ) stomach_pain
2 ) pain_behind_the_eyes
3 ) back_pain
4 ) abdominal_pain
5 ) chest_pain
6 ) pain_during_bowel_movements
7 ) pain_in_anal_region
8 ) neck_pain
9 ) knee_pain
10 ) hip_joint_pain
11 ) muscle_pain
12 ) belly_pain
13 ) painful_walking
Select the one you meant (0 - 13):                                                                                                                   Predicted disease at this step: Alcoholic hepatitis
Are you experiencing any:
vomiting? : Current prediction based on your symptoms: Alcoholic hepatitis or (vertigo) Paroymsal  Positional Vertigo
yellowish_skin? : Current prediction based on your symptoms: Alcoholic hepatitis or Chronic cholestasis
abdominal_pain? : Current prediction based on your symptoms: Alcoholic hepatitis or Chronic cholestasis
swelling_of_stomach? : Current prediction based on your symptoms: Alcoholic hepatitis or Alcoholic hepatiti

In [25]:
pip install joblib


Note: you may need to restart the kernel to use updated packages.


In [26]:
import joblib

# Save the model to a .pkl file
model_filename = 'decision_tree_model.pkl'
joblib.dump(clf, model_filename)

print(f"Model saved as {model_filename}")


Model saved as decision_tree_model.pkl


In [27]:
import pickle


# Example model creation
model = clf
# Train your model here
# model.fit(X_train, y_train)

# Save the model
with open('decision_tree2_model.pkl', 'wb') as f:
    pickle.dump(model, f)
